In [ ]:
%%writefile requirements.txt
lime
keras
pandas
tensorflow

In [ ]:
!pip install -r requirements.txt

In [ ]:
pip install scikeras[tensorflow]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dati/dataset_taggato.csv', sep = ';')

In [ ]:
print(df.head(1))

In [ ]:
x = df.iloc[: , 1:16]
cols = x.columns
x[cols] = x[cols].replace(',','.', regex = True)
x[cols] = x[cols].apply(pd.to_numeric,downcast='float', errors='raise')
x = x.astype('float64')

scaler = StandardScaler()

x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)

In [ ]:
ids = df["spotify_id"]

In [ ]:
x.insert(0, "spotify_id", ids.values)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
y = df.iloc[: , 28:30]
cols = y.columns
y[cols] = y[cols].replace(',','.', regex = True)

y[cols] = y[cols].apply(pd.to_numeric,downcast='float', errors='raise')

y = pd.DataFrame(scaler2.fit_transform(y.values), columns=y.columns, index=y.index)
print(y.head(5))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/nn_model")

In [ ]:
predictions = model.predict(x_test)

In [ ]:
l = 2000
print(predictions[l])
print(y_test.iloc[l])

In [ ]:
def get_predict_fn(idx):
  def predict_fn(x_test):
    return model.predict(x_test)[:, idx]
  return predict_fn

#PASTLE

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/PASTLE')
import pivots_generators
import explainers
import explanations

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dati/dataset_taggato.csv', sep=',')
df = df.drop('index', axis=1)
x = df.iloc[:, 0:15].to_numpy()
y = df.iloc[:, 15:17].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

proximity_fns = {
    'p1': (lambda xx: 1 / (1 + xx)),
    'p2': (lambda xx: np.exp(-xx)),
    'p3': (lambda xx: -xx),
    'p4': (lambda xx: 1 - (xx - np.min(xx, axis=0)) / (np.max(xx, axis=0) - np.min(xx, axis=0))),
    'p5': (lambda xx: np.max(xx, axis=0) - xx),
}

feature_names = df.columns[:-2].tolist()
class_names = ['valence_tags', 'arousal_tags']
dataset = df.iloc[:, :-2].to_numpy()
labels = df.iloc[:, 15:17].to_numpy()

proximity_fn = proximity_fns['p1']

n_pivots = 25
n_features = x_train.shape[1]
n_samples = 150
distance_fn = 'euclidean'
pivots_generator = pivots_generators.Random(x_train, n_pivots)

#Definizione numero di pivot


In [ ]:
punteggi = []
kernel_width = 0.71
n_samp = 100
n_pivots = 25
pivots_generator = pivots_generators.Random(x_train, n_pivots)

explainer = explainers.TabularExplainer(x_train, cluster_model=pivots_generator, mode="regression", feature_names=feature_names, 
                                        class_names=class_names, discretize_continuous=False, sample_around_instance=True, 
                                        verbose=False, proximity_function=proximity_fn, kernel_width = kernel_width)

for k in range (5):
  temp1 = []
  temp2 = []
  exp1 = explainer.explain_instance(x_test[l],
                                 predict_fn = get_predict_fn(0),
                                 top_labels=1,
                                 num_pivots=n_pivots,
                                 num_samples=n_samp,
                                 verbose=False,
                                 distance_metric=distance_fn,
                                 model_regressor=LinearRegression())
  exp2 = explainer.explain_instance(x_test[l],
                                 predict_fn = get_predict_fn(1),
                                 top_labels=1,
                                 num_pivots=n_pivots,
                                 num_samples=n_samp,
                                 verbose=False,
                                 distance_metric=distance_fn,
                                 model_regressor=LinearRegression())
  temp1.append(exp1[0].score)
  temp2.append(exp2[0].score)
  temp1 = np.asarray(temp1)
  temp2 = np.asarray(temp2)
  media_1 = np.mean(temp1, dtype=np.float64)
  media_2 = np.mean(temp2, dtype=np.float64)
  punteggi.append([media_1, media_2])



In [ ]:
from sklearn.linear_model import LinearRegression

#Valutazione larghezza kernel e numero di vicini

In [ ]:
import time
from numpy import savetxt

start_time = time.time()

scores = []
for l in range(1000):
  
  for i in np.arange(0.01, 1, 0.05): 
    print('Kernel ='+ str(i))

    explainer = explainers.TabularExplainer(x_train,
                             cluster_model=pivots_generator,
                             mode="regression",
                             feature_names=feature_names,
                             class_names=class_names,
                             discretize_continuous=False,
                             sample_around_instance=True,
                             verbose=False,
                             proximity_function=proximity_fn,
                             kernel_width = i)
    for f in range (50, 250, 50):
      v = []
      a = []
      for h in range(5):

        exp1 = explainer.explain_instance(x_test[l], predict_fn = get_predict_fn(0), num_features=15, num_samples=f, model_regressor=LinearRegression())
        exp2 = explainer.explain_instance(x_test[l], predict_fn = get_predict_fn(1), num_features=15, num_samples=f, model_regressor=LinearRegression())
        v.append(exp1[0].score)
        a.append(exp2[0].score)

      v = np.asarray(v)
      a = np.asarray(a)
      media_v = np.mean(v, dtype=np.float64)
      media_a = np.mean(a, dtype=np.float64)
      print("Numero di Vicini: "+str(f))  
      print('['+str(media_v) +', '+ str(media_a)+']')
    
      score = [media_v, media_a]
      scores.append(score)
      scores.append([123456789, 123456789])



savetxt('/content/drive/MyDrive/Colab Notebooks/PastleLineardata.csv', scores, delimiter=',')

#Valutazione contributo pivot

In [ ]:
def feature_finder(s):
  char = "" 
  for x in s: 
    if x.isalpha(): 
        char = "".join([char, x])
  return char

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_taggato.csv', sep = ';')

#display(df)

In [ ]:
x = df.iloc[: , 1:16]
cols = x.columns
x[cols] = x[cols].replace(',','.', regex = True)
x[cols] = x[cols].apply(pd.to_numeric,downcast='float', errors='raise')
x = x.astype('float64')

scaler = StandardScaler()

x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)

#display(x)

#print(x.dtypes)

In [ ]:
Pivots = ['Pivot_0', 'Pivot_1', 'Pivot_2', 'Pivot_3', 'Pivot_4', 'Pivot_5', 'Pivot_6', 'Pivot_7', 'Pivot_8', 'Pivot_9', 'Pivot_10',
          'Pivot_11', 'Pivot_12', 'Pivot_13', 'Pivot_14', 'Pivot_15', 'Pivot_16', 'Pivot_17', 'Pivot_18', 'Pivot_19', 'Pivot_20', 
          'Pivot_21', 'Pivot_22', 'Pivot_23', 'Pivot_24']
features = Pivots
Pastle_dictionary_v_p = { feature : 0 for feature in features }
Pastle_dictionary_v_n = { feature : 0 for feature in features }
Pastle_dictionary_a_p = { feature : 0 for feature in features }
Pastle_dictionary_a_n = { feature : 0 for feature in features }


In [ ]:
val = []
ar = []
kernel_width = 0.71
n_samp = 150
explainer = explainers.TabularExplainer(x_train, cluster_model=pivots_generator, mode="regression", feature_names=feature_names, 
                                        class_names=class_names, discretize_continuous=False, sample_around_instance=True, 
                                        verbose=False, proximity_function=proximity_fn, kernel_width = kernel_width)
for i in range(1000):
  #print(str(i)+' di 1000')
  expv = explainer.explain_instance(x_test[i], predict_fn = get_predict_fn(0), top_labels=1, num_pivots=n_pivots,num_samples=n_samp, 
                                    verbose=False, distance_metric=distance_fn, model_regressor=LinearRegression())
  exp1 = expv[0]
  expa = explainer.explain_instance(x_test[i], predict_fn = get_predict_fn(1), top_labels=1, num_pivots=n_pivots,num_samples=n_samp,
                                    verbose=False, distance_metric=distance_fn, model_regressor=LinearRegression())
  exp2 = expa[0]
  li1 = exp1.as_list()
  li2 = exp2.as_list()
  val.append(exp1.score)
  ar.append(exp2.score)
  
  for lun in range(len(li1)):
    valore = li1[lun][1]
    feature = li1[lun][0]
    if valore >0:
      Pastle_dictionary_v_p[feature] = Pastle_dictionary_v_p[feature]+valore #1
    else:
      Pastle_dictionary_v_n[feature] = Pastle_dictionary_v_n[feature]+valore #1
    

  for lun in range(len(li2)):
    valore = li2[lun][1]
    feature = li2[lun][0]
    if valore >0:
      Pastle_dictionary_a_p[feature] = Pastle_dictionary_a_p[feature]+valore #1
    else:
      Pastle_dictionary_a_n[feature] = Pastle_dictionary_a_n[feature]+valore #1
    
  
print(Pastle_dictionary_v_p)
print(Pastle_dictionary_v_n)
print(Pastle_dictionary_a_p)
print(Pastle_dictionary_a_n)

In [ ]:
Valence = []
Arousal = []
for i in range(len(val)):
  Valence.append([val[i]])
  Arousal.append([ar[i]])
  print(str(val[i])+str(ar[i]))

val = np.array(Valence)
ar = np.array(Arousal)

print(np.mean(val))
print(np.mean(ar))

In [ ]:
import json 
      
with open("/content/drive/MyDrive/Colab Notebooks/Pastle_dictionary_v_n.json", "w") as outfile:
    json.dump(Pastle_dictionary_v_n, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/Pastle_dictionary_v_p.json", "w") as outfile:
    json.dump(Pastle_dictionary_v_p, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/Pastle_dictionary_a_p.json", "w") as outfile:
    json.dump(Pastle_dictionary_a_p, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/Pastle_dictionary_a_n.json", "w") as outfile:
    json.dump(Pastle_dictionary_a_n, outfile)

#Explain instance

In [ ]:
kernel_width = 0.71
n_samp = 150
explainer = explainers.TabularExplainer(x_train, cluster_model=pivots_generator, mode="regression", feature_names=feature_names, 
                                        class_names=class_names, discretize_continuous=False, sample_around_instance=True,
                                        verbose=False, proximity_function=proximity_fn, kernel_width = kernel_width)
expv = explainer.explain_instance(x_test[l], predict_fn = get_predict_fn(0), top_labels=1, num_pivots=n_pivots, num_samples=n_samp,
                                  verbose=False, distance_metric=distance_fn, model_regressor=LinearRegression())
exp1 = expv[0]
expa = explainer.explain_instance(x_test[l], predict_fn = get_predict_fn(1), top_labels=1, num_pivots=n_pivots,num_samples=n_samp,
                                  verbose=False, distance_metric=distance_fn, model_regressor=LinearRegression())
exp2 = expa[0]

print(str(exp1.score)+' '+str(exp2.score))

In [ ]:
feature_importances =  expv[1].local_exp[1] 
#true_pred = model.predict(x_test[l])
pivots = pivots_generator.pivots
mode = 'regression'
new_exp = explanations.PASTLE_Explanation(
                 x_train,
                 y_train[:,0],
                 expv[0],  # LIME explanation object
                 pivots,
                 feature_importances,
                 x_test[l],
                 predictions[l][0],  # black-box prediction
                 feature_names,
                 mode = mode,
                 verbose=False
                 )

new_exp.show_in_notebook(save=True,name='regression.pdf')

In [ ]:
pts_supporting, preds_supporting, pts_opposing, preds_opposing = new_exp.move_along_directions(model,n_points=300,save=True,name='regression_directions.pdf', verbose=False)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
stop_point = len(pts_supporting)
ax.tick_params(bottom=False, labelsize=12)
ax.set_xticks([])
ax.set_xlabel('Points along directions',size=24)
ax.set_ylabel('Model prediction',size=24)
xx = range(stop_point)
yy = preds_supporting[:stop_point, 0]
ax.plot(xx, yy, color='#238823')
yy = preds_opposing[:stop_point, 0]
ax.plot(xx, yy, color='#E42531')
plt.grid(True)
plt.legend(['Positive direction', 'Negative direction'],prop={'size': 24})

In [ ]:
feature_importances =  expa[1].local_exp[1] 
#true_pred = model.predict(x_test[l])
pivots = pivots_generator.pivots
mode = 'regression'
new_exp = explanations.PASTLE_Explanation(
                 x_train,
                 y_train[:,1],
                 expa[0],  # LIME explanation object
                 pivots,
                 feature_importances,
                 x_test[l],
                 predictions[l][1],  # black-box prediction
                 feature_names,
                 mode = mode,
                 verbose=False
                 )

new_exp.show_in_notebook(save=True,name='regression.pdf')

In [ ]:
pts_supporting, preds_supporting, pts_opposing, preds_opposing = new_exp.move_along_directions(model,n_points=300,save=True,name='regression_directions.pdf', verbose=False)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
stop_point = len(pts_supporting)
ax.tick_params(bottom=False, labelsize=12)
#ax.set_xticks([])
ax.set_xlabel('Points along directions',size=24)
ax.set_ylabel('Model prediction',size=24)
xx = range(stop_point)
yy = preds_supporting[:stop_point, 1]
ax.plot(xx, yy, color='#238823')
yy = preds_opposing[:stop_point, 1]
ax.plot(xx, yy, color='#E42531')
plt.grid(True)
plt.legend(['Positive direction', 'Negative direction'],prop={'size': 24})

In [ ]:
print(x_train.shape)
print(len(x_train))
print(pts_supporting.shape)
print(len(pts_supporting))

In [ ]:
from datetime import datetime

suggestions = []
mins = []

start_time = datetime.now()
print("Current Time =", start_time)
min = 100000000
for i in range(len(x_train)-1):
  #print(str(i))
  for l in range((30)):
    dist = np.linalg.norm(x_train[i]-pts_supporting[l])
    if dist <= min: 
      min = dist
      suggestions.append([i,l])
      mins.append(min)
now = datetime.now()
end_time = datetime.now()
print("Current Time =", end_time)
print('Duration: {}'.format(end_time - start_time))